In [1]:
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

In [2]:
ID="20220225_131619_robustness"
ID

'20220225_131619_robustness'

In [4]:
rm -rf /data/output/$ID

In [5]:
mkdir /data/output/$ID

In [6]:
cp meta.csv /data/output/$ID

In [18]:
ls -lath /data/output/$ID/logs/

total 312K
drwxr-xr-x. 2 root root 4.0K Feb 25 13:24 ./
-rw-r--r--. 1 root root   56 Feb 25 13:24 pipeline_log.txt
-rw-r--r--. 1 root root  910 Feb 25 13:24 20220225_132319_extract.log
drwxr-xr-x. 3 root root 4.0K Feb 25 13:23 ../
-rw-r--r--. 1 root root 284K Feb 25 13:22 20220225_132136_create.log
-rw-r--r--. 1 root root  700 Feb 25 13:20 20220225_132027_create.log
-rw-r--r--. 1 root root 7.3K Feb 25 13:18 20220225_131809_create.log


In [21]:
cat  /data/output/$ID/logs/20220225_132319_extract.log

2022-02-25 13:23:19,028 [INFO]  starting extraction Namespace(dir='/data/output/20220225_131619_full/', filter_info_output_file='/data/output/20220225_131619_full/filterinfo.csv', filter_output_file='/data/output/20220225_131619_full/', filter_shape=(3, 3), first_convs_only=False, meta_file=<_io.TextIOWrapper name='/data/output/20220225_131619_full/datastorage.meta.csv' mode='r' encoding='UTF-8'>)
2022-02-25 13:23:19,031 [INFO]  loading chunk /data/output/20220225_131619_full/datastorage.0000.pkl
2022-02-25 13:23:34,475 [INFO]  loaded ... processing
2022-02-25 13:23:52,473 [INFO]  finished chunk /data/output/20220225_131619_full/datastorage.0000.pkl with 117 models
2022-02-25 13:23:54,926 [INFO]  writing filter info dataframe: /data/output/20220225_131619_full/filterinfo.csv
2022-02-25 13:23:54,941 [INFO]  writing filters /data/output/20220225_131619_full/
2022-02-25 13:24:41,766 [INFO]  finished


In [15]:
!cd onnxzoo2filterdb && python3 create_filter_storage.py /ssd1/robustness_onnx_zoo/ /data/output/$ID/meta.csv /data/output/$ID/ --split_after 1000 --force

100%|█████████████████████████████████████████| 117/117 [01:39<00:00,  1.18it/s]


In [16]:
!cd onnxzoo2filterdb && python3 extract_filters.py /data/output/$ID/ /data/output/$ID/datastorage.meta.csv /data/output/$ID/filterinfo.csv /data/output/$ID/

2022-02-25 13:23:19,028 [INFO]  starting extraction Namespace(dir='/data/output/20220225_131619_full/', filter_info_output_file='/data/output/20220225_131619_full/filterinfo.csv', filter_output_file='/data/output/20220225_131619_full/', filter_shape=(3, 3), first_convs_only=False, meta_file=<_io.TextIOWrapper name='/data/output/20220225_131619_full/datastorage.meta.csv' mode='r' encoding='UTF-8'>)
2022-02-25 13:23:19,031 [INFO]  loading chunk /data/output/20220225_131619_full/datastorage.0000.pkl
2022-02-25 13:23:34,475 [INFO]  loaded ... processing
2022-02-25 13:23:52,473 [INFO]  finished chunk /data/output/20220225_131619_full/datastorage.0000.pkl with 117 models
2022-02-25 13:23:54,926 [INFO]  writing filter info dataframe: /data/output/20220225_131619_full/filterinfo.csv
2022-02-25 13:23:54,941 [INFO]  writing filters /data/output/20220225_131619_full/
2022-02-25 13:24:41,766 [INFO]  finished


In [17]:
!echo "Created by processing_pipeline.ipynb on" $timestamp >> /data/output/$ID/logs/pipeline_log.txt

In [10]:
from onnxzoo2filterdb.filter_dataset import FilterDataset
import pandas as pd
import numpy as np
import os

root_path = "/data/output"
dataset_id = ID
filter_dataset = FilterDataset(dataset_id, root=root_path, filters=False)
fid_by_mid_with_meta = filter_dataset.df_filter_info.groupby([filter_dataset.df_filter_info.index, filter_dataset.df_filter_info.conv_depth, filter_dataset.df_filter_info.conv_depth_norm, filter_dataset.df_filter_info.layer_id]).filter_ids.apply(lambda x: np.hstack(x)).to_frame().join(filter_dataset.df_fused_meta, on=["model_id"])
fid_by_mid_with_meta["filter_ids"] = fid_by_mid_with_meta["filter_ids"].apply(lambda x: f"{x[0]}:{x[-1]}")
fid_by_mid_with_meta.to_hdf(os.path.join(root_path, dataset_id, "dataset.h5"), "meta")

Loading tables ...
 df_filter_info
 df_meta
 df_meta_spreadsheets
 merging meta
Done


/tmp/ipykernel_91636/73617082.py:11: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['filter_ids', 'model', 'path', 'producer', 'Name', 'Paper',
       'Tracer Warning', 'Selection', 'Robust', 'Network', 'Backbone',
       'Framework', 'Pretraining-Dataset', 'Training-Dataset',
       'Visual Category', 'Visual Category_micro', 'Precision', 'Task'],
      dtype='object')]

  fid_by_mid_with_meta.to_hdf(os.path.join(root_path, dataset_id, "dataset.h5"), "meta")
